In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class neural_net:
    
    def __init__(self,x,y,struct,activ,epoch,lr=0.01):
        self.layers=len(struct)
        self.struct=struct
        self.activ=activ
        self.x=x
        self.y=y
        self.param=self.evaluate(epoch,lr,cost_out= False)

    def gen_network(self):
        param={}
        for i in range(self.layers):
            if i==0:a,b=self.struct[i],self.x.shape[0]
            else:a,b=self.struct[i],self.struct[i-1]
            param[f'w{i+1}']=np.random.randn(a,b)
            param[f'b{i+1}']=np.random.randn(a,1)
        return param

    def activate(self,z,func,d=False,alpha=0.01):
        if func=='s':
            if not d: return 1/(1+np.exp(-z))
            else:     return z*(1-z)
        elif func=='t':
            if not d: return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
            else:     return 1-z**2
        elif func=='r':
            if not d: return np.maximum(0,z)
            else:     return np.int64(z>0)
        elif func=='lr':
            if not d: return np.maximum(0.01*z,z)
            else:     return np.where(z>0,1,alpha)
            
    def forward(self,param):
        fwd={}
        for i in range(self.layers):
            if i==0:
                fwd[f'z{i+1}']=np.dot(param['w1'],self.x)+param[f'b{i+1}']
                fwd[f'a{i+1}']=self.activate(z=(fwd[f'z{i+1}']),func=self.activ[i]) 
            else:
                fwd[f'z{i+1}']=np.dot(param[f'w{i+1}'],fwd[f'a{i}'])+param[f'b{i+1}']
                fwd[f'a{i+1}']=self.activate(z=fwd[f'z{i+1}'],func=self.activ[i]) 
        return fwd

    def back(self,fwd,param):
        delta={}
        for i in range(self.layers,0,-1):
            if i==self.layers:
                delta[f'dz{i}']=fwd[f'a{i}']-self.y
                delta[f'dw{i}']=np.dot(delta[f'dz{i}'],fwd[f'a{i-1}'].T)/self.x.shape[1]
                delta[f'db{i}']=np.sum(delta[f'dz{i}'],axis=1,keepdims=True)/self.x.shape[1]
            elif i==1:
                delta[f'dz{i}']=(np.dot(param[f'w{i+1}'].T,delta[f'dz{i+1}']))*(self.activate(fwd[f'z{i}'],self.activ[self.layers-i-1],d=True))
                delta[f'dw{i}']=np.dot(delta[f'dz{i}'],self.x.T)/self.x.shape[1]
                delta[f'db{i}']=np.sum(delta[f'dz{i}'],axis=1,keepdims=True)/self.x.shape[1]
            else:
                delta[f'dz{i}']=(np.dot(param[f'w{i+1}'].T,delta[f'dz{i+1}']))*(self.activate(fwd[f'z{i}'],self.activ[self.layers-i-1],d=True))
                delta[f'dw{i}']=np.dot(delta[f'dz{i}'],fwd[f'a{i-1}'].T)/self.x.shape[1]
                delta[f'db{i}']=np.sum(delta[f'dz{i}'],axis=1,keepdims=True)/self.x.shape[1]
        return delta

    def get_accuracy(self,yh):
        count=0
        for i in range(self.y.shape[1]):
            if self.y[0][i]!=yh[0][i]:count+=1
        return float((self.y.shape[1]-count)/self.y.shape[1])*100
    
    def update(self,param,delta,lr):
        for i in param:
            param[i]-=lr*delta[f'd{i}']
        return param
    
    def cost(self,yh,n):
        m = self.y.shape[1]
        try:
            logp = np.multiply(self.y,np.log(yh))+np.multiply((1-self.y),np.log(1-yh))
            cost = np.sum(-logp)/m
        except Exception as e:
            print(n,e)
        cost=float(np.squeeze(cost))
        return cost
    
    def predict(self):
        pred=self.forward(self.param)
        yh=(pred[f'a{self.layers}']>0.5)
        # print('Accuracy: ',self.get_accuracy(yh))
        return self.get_accuracy(yh)
    
    def prnt_dict(self,dict):
        for i in dict:
            print(i,dict[i])
            print()

    def evaluate(self,epoch,lr,cost_out=True):  
        param=self.gen_network()
        # self.prnt_dict(param)
        cost=[]
        for i in range(epoch):
            fwd=self.forward(param=param)
            cost.append(self.cost(fwd[f'a{len(self.struct)}'],i))
            delta=self.back(fwd,param)
            param=self.update(param,delta,lr)
        if cost_out:
            plt.title(f'lr: {lr}')
            plt.plot(cost)
        # self.prnt_dict(param)
        return param

In [ ]:
import numpy as np
x=np.random.randn(40,100)
y=np.random.randn(1,100)>0
nn=neural_net(x,y,[7,6,5,4,3,2,1],['r','r','r','r','r','lr','s'],5000,00.01)
nn.predict()

# lrs=np.arange(0.001,0.1,0.001)
# mx=[0,0]
# for i in lrs:
#     nn=neural_net(x,y,[7,6,5,4,3,2,1],['r','r','r','r','r','lr','s'],5000,i)
#     acc=round(nn.predict(),4)
#     if mx[1]<acc and acc!=float(100):
#         mx[0]=i
#         mx[1]=acc
#         print(round(i,4),round(nn.predict(),4))